In [40]:
from langchain_ollama.llms import OllamaLLM
from dotenv import load_dotenv
import streamlit as st
from PIL import Image
import io
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

load_dotenv()

MODEL_NAME = os.getenv("MODEL_NAME")
llm = ChatOllama(
    model=MODEL_NAME,
    temperature=0.3,
    # other params...
)
llm.format = "json"

In [38]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a agent that creates google calendar events. Your output shoould be in the following format:event = {{
      "summary": " ",
      "location": " ",
      "description": " ",
      "start": {{
        "dateTime": " ",
        "timeZone": "America/Los_Angeles",
      }},
      "end": {{
        "dateTime": " ",
        "timeZone": "America/Los_Angeles",
      }},
      "recurrence": [
        "RRULE:FREQ=DAILY;COUNT=1"
      ],
      "attendees": [
        {{"email": " "}},
      ],
      "reminders": {{
        "useDefault": False,
        "overrides": [
          {{"method": "email", "minutes": 24 * 60}},
          {{"method": "popup", "minutes": 10}},
        ],
      }},
    }} 
    From the user input, see if you can extract relevant information and for information that is missing, you can ask the user questions that you think are necessary to create the event. Remember to cause least friction to the user. Only ask information with technical details, any details that can be inferred from the user input, should be inferred geneously.
    """,
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
a = chain.invoke({ 
        "input":"Can you create an event on 5th February 2025 11:00 AM - 2:00 PM to attend a conference at Unviersity of Ottawa with no attendees?"})

In [39]:
print(a.content)

<think>
Okay, so I need to figure out how to create a Google Calendar event based on the user's request. Let me break down what they're asking for.

First, the event needs to be on February 5th, 2025, from 11:00 AM to 3:00 PM. The location is the University of Ottawa, and there are no attendees required. They also want reminders set up so that it's not missed.

Looking at the structure they provided, I see fields like summary, location, description, start and end times with time zones, recurrence, attendees, and reminders. 

I notice that in their example response, they filled in most of these fields but left some placeholders for me to ask about if information was missing. So, my job is to check each field and see if I can extract the necessary details from the user's input.

The event summary isn't provided yet, so I should probably set a default like "University of Ottawa Conference" since that's what they mentioned in their request.

For location, it's specified as the University o

In [50]:
from typing import List
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from event_handler import create_event

MODEL_NAME = os.getenv("MODEL_NAME")
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0.3,
    # other params...
).bind_tools([create_event])
# llm.format = "json"

result = llm.invoke(
    "Can you create a google event on 5th February 2025 11:00 AM - 2:00 PM to attend a conference at Unviersity of Ottawa with no attendees?"
)
result.tool_calls

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=672071315280-eh79l4fu8iplnfupom24u5rnq37msdag.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56380%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=8As7chXBLDbDg076LzmjCkZMLqB6km&access_type=offline


[{'name': 'create_event',
  'args': {'attendees': [''],
   'end_time': '2025-02-05T14:00:00-05:00',
   'location': 'University of Ottawa',
   'start_time': '2025-02-05T11:00:00-05:00'},
  'id': '45859898-ec82-4a3b-badc-c2e15a6c1994',
  'type': 'tool_call'}]

In [47]:
print(result)

content='' additional_kwargs={} response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-01-25T22:27:58.192299Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4071359750, 'load_duration': 820197584, 'prompt_eval_count': 303, 'prompt_eval_duration': 1568000000, 'eval_count': 70, 'eval_duration': 1677000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-2a4a6c16-e940-45e7-9c11-f9aa47bde46b-0' tool_calls=[{'name': 'create_event', 'args': {'attendees': [], 'end_time': '2025-02-05T14:00:00-05:00', 'location': 'University of Ottawa', 'start_time': '2025-02-05T11:00:00-05:00'}, 'id': 'e9371dc8-9908-4d75-bc8f-1b91f692f928', 'type': 'tool_call'}] usage_metadata={'input_tokens': 303, 'output_tokens': 70, 'total_tokens': 373}


In [51]:
## Check the Langgraph Imlementation of the ReAct Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
print(create_react_agent())

TypeError: create_react_agent() missing 2 required positional arguments: 'model' and 'tools'